In [1]:
import torchaudio


In [2]:
def load_tags(file_path):
    tag_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            tag, code = line.strip().split()
            tag_dict[code] = tag
    return tag_dict

In [3]:
tag_dict = load_tags('/external2/datasets/text/synthetic/processed_r3/alltags.txt')

In [133]:
import nemo
import torch
import nemo.collections.asr as nemo_asr

map_location = torch.device('cpu')

asr_model = nemo_asr.models.ASRModel.restore_from("/external2/karan_exp/experiments/finetune-alllang-130k-ctc-r3/2024-07-22_01-37-14/checkpoints/finetune-alllang-130k-ctc-r3.nemo", map_location=map_location)
# Set the model to evaluation mode

asr_model.eval()

[NeMo I 2024-07-22 10:57:40 mixins:172] Tokenizer SentencePieceTokenizer initialized with 2500 tokens


[NeMo W 2024-07-22 10:57:41 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /external2/datasets/text/synthetic/processed_r3/train.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2024-07-22 10:57:41 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /external2/datasets/text/synthetic/processed_r3/valid.json
    sample_rate: 16000
    batch_size: 16
    shuf

[NeMo I 2024-07-22 10:57:41 features:289] PADDING: 0
[NeMo I 2024-07-22 10:57:43 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /external2/karan_exp/experiments/finetune-alllang-130k-ctc-r3/2024-07-22_01-37-14/checkpoints/finetune-alllang-130k-ctc-r3.nemo.


EncDecCTCModelBPE(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConformerEncoderAdapter(
    (pre_encode): ConvSubsampling(
      (out): Linear(in_features=10240, out_features=512, bias=True)
      (conv): Sequential(
        (0): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (3): ReLU(inplace=True)
      )
    )
    (pos_enc): RelPositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-17): 18 x ConformerLayer(
        (norm_feed_forward1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (feed_forward1): ConformerFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (activation): Swish()
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features

In [134]:
# Define a function to map tags to words
def map_tags(word, tag_dict):
    parts = word.split('_') if '_' in word else word.split('-')
    mapped_parts = [tag_dict.get(part, part) for part in parts]
    return ' '.join(mapped_parts)

# Define a function to transcribe the audio
def transcribe(audio_file):
    # Load your 16k wav audio file
    audio, sample_rate = torchaudio.load(audio_file)

    if sample_rate != 16000:
        raise ValueError(f"Expected sample rate of 16000, but got {sample_rate}")

    # Perform transcription
    with torch.no_grad():
        transcription = asr_model.transcribe([audio_file])[0]

    # Map the tags to the corresponding words
    print(transcription)
    words = transcription.split()
    mapped_transcription = ' '.join([map_tags(word, tag_dict) for word in words])

    return mapped_transcription

In [167]:
# Transcribe your audio file
audio_file = '/home/ksingla/workspace/PromptingNemo/applications/voicebot/demo_audio/librespeech_4.wav'
print(f"Transcription: {transcribe(audio_file)}")


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

T1 T2 firstthing Ider of them notied a new soundd brok into the clean alourmh ever herit beforeballall of the mony of T237_ onfilage rreing for arly preraers. T111_T122 T0
Transcription: TASK TAGGER firstthing Ider of them notied a new soundd brok into the clean alourmh ever herit beforeballall of the mony of ENTITY  onfilage rreing for arly preraers. INTENT COMMENT EOS


In [168]:
from IPython.display import Audio

#audio_file = '/external2/datasets/LibriSpeech/test-converted-wav/1089-134686-0023.wav'
Audio(audio_file)

In [ ]:
Transcription: DOMAIN SOCCER HIGHLIGHTS ENTITY PLAYER  was END the screment END of the ENTITY DEVICE ukaristu instuted underter the2o speies of bread and wine if jeeseuse cricet be presvanentt body and blood sol and divnity in the bread alonan and in theT813 wine a loan. INTENT ANALYSIS COMMENT

In [ ]:
from google.cloud import texttospeech
import os

# Set up your Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/ksingla/workspace/medical-ner/keys/google-tts-key.json"

def list_available_voices():
    # Initialize the Text-to-Speech client
    client = texttospeech.TextToSpeechClient()

    # Performs the list voices request
    response = client.list_voices()

    # Initialize a dictionary to store voices by language code
    voice_dict = {
        'EN': [], 'ES': [], 'FR': [], 'DE': [], 'IT': [], 'PT': [], 'NL': [], 'SV': []
    }

    # Define the language code mapping
    language_mapping = {
        'EN': 'en', 'ES': 'es', 'FR': 'fr', 'DE': 'de',
        'IT': 'it', 'PT': 'pt', 'NL': 'nl', 'SV': 'sv'
    }

    # Populate the dictionary with available voices
    for voice in response.voices:
        for language_code in voice.language_codes:
            for key, value in language_mapping.items():
                if language_code.startswith(value):
                    voice_dict[key].append(voice.name)

    return voice_dict

# List all available voices
voices = list_available_voices()

In [ ]:
voices

In [ ]:
from langdetect import detect, detect_langs

# Sample sentence
sentence = "나는 몸살이 나 오늘 아침 있어요"

# Detect language
language = detect(sentence)

# Detect language probabilities
languages = detect_langs(sentence)

print(f"The detected language is: {language}")
print(f"Language probabilities: {languages}")
